In [ ]:
# %% [markdown]
# # Emergent Machine Pedagogy: A Game-Theoretic Framework
# 
# This notebook provides a complete, end-to-end pipeline to run the experiments from the dissertation on a Google Colab T4 GPU.
# 
# ### Instructions:
# 1.  **Set Runtime Type:** Go to `Runtime` > `Change runtime type` and select `T4 GPU`.
# 2.  **Upload Project:**
#     *   On your local machine, zip your entire `cognitive-architectures` project folder. **Important:** Make sure to exclude the large `models/` directory from the zip file to keep the upload small.
#     *   In the Colab file browser on the left, click the "Upload to session storage" button and upload your `cognitive-architectures.zip` file.
# 3.  **Run All Cells:** Go to `Runtime` > `Run all`. The notebook will set up the environment, download the necessary models, and run the full experimental suite.

# %% [markdown]
# ---
# ### Cell 1: Environment Setup & Project Unzip
# 
# This cell prepares the Colab environment. It unzips the project, installs the correct GPU-accelerated dependencies, and sets up the necessary environment variables for the scripts.

# %%
import os
import subprocess

# --- Unzip the project folder ---
# This assumes you have uploaded "cognitive-architectures.zip" to the Colab session.
PROJECT_ZIP_NAME = "cognitive-architectures.zip"
if not os.path.exists(PROJECT_ZIP_NAME):
    print(f"ERROR: Please upload '{PROJECT_ZIP_NAME}' to the Colab session storage.")
else:
    print(f"Unzipping {PROJECT_ZIP_NAME}...")
    subprocess.run(["unzip", "-q", PROJECT_ZIP_NAME])
    print("Project unzipped successfully.")



In [ ]:
# --- Define project root and move into it ---
PROJECT_ROOT = "cognitive-architectures"
os.chdir(PROJECT_ROOT)
print(f"Changed current working directory to: {os.getcwd()}")




In [ ]:
# --- Install GPU-accelerated dependencies ---
# We use the requirements.colab.txt file which specifies CUDA compilation.
print("\nInstalling GPU-accelerated dependencies from requirements.colab.txt...")
# The CMAKE_ARGS tell llama-cpp-python to build with CUDA support (cuBLAS).
# This is the key step for GPU acceleration.
os.environ["CMAKE_ARGS"] = "-DLLAMA_CUBLAS=on"
os.environ["FORCE_CMAKE"] = "1"
subprocess.run(["pip", "install", "-r", "requirements.colab.txt"])
print("Dependencies installed successfully.")




In [ ]:
# --- Set the PYTHONPATH ---
# This ensures that `from src...` imports work correctly from the experiments directory.
os.environ["PYTHONPATH"] = os.getcwd()
print(f"PYTHONPATH set to: {os.environ['PYTHONPATH']}")


# --- Download the main LLM model ---
print("\nDownloading the Phi-3 GGUF model...")
MODEL_DIR = "models"
MODEL_NAME = "Phi-3-mini-4k-instruct-q4.gguf"
MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME)

if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_DIR, exist_ok=True)
    subprocess.run([
        "wget", 
        "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf",
        "-O", MODEL_PATH
    ])
    print("Model downloaded successfully.")
else:
    print("Model already exists.")



In [ ]:
# %% [markdown]
# ---
# ### Cell 2: Verify GPU and Modify Code for GPU Execution
# 
# This cell verifies that the GPU is available and makes a crucial runtime modification to the `src/common.py` file to enable full GPU offloading.

# %%
import torch

# --- Verify GPU Access ---
if torch.cuda.is_available():
    print("✅ GPU is available. PyTorch can see the T4.")
    !nvidia-smi
else:
    print("❌ WARNING: GPU not found. The experiments will be extremely slow.")




In [ ]:
# --- Runtime Code Patch for GPU ---
# This is a critical step. We programmatically change n_gpu_layers from 0 to -1
# to ensure the model is fully offloaded to the T4 GPU.
common_py_path = "src/common.py"
print(f"\nPatching {common_py_path} for full GPU offloading...")

with open(common_py_path, 'r') as f:
    lines = f.readlines()

new_lines = []
patched = False
for line in lines:
    if "n_gpu_layers=" in line:
        new_line = '    n_gpu_layers=-1, # <-- Patched by Colab notebook for full GPU offloading\n'
        new_lines.append(new_line)
        print(f"  - Found and replaced line: {line.strip()}")
        print(f"  + With new line: {new_line.strip()}")
        patched = True
    else:
        new_lines.append(line)

if patched:
    with open(common_py_path, 'w') as f:
        f.writelines(new_lines)
    print("Patching successful.")
else:
    print("Warning: Could not find 'n_gpu_layers' line to patch.")



In [ ]:
# %% [markdown]
# ---
# ### Cell 3: Run the Dry-Run Experiment
# 
# This runs a quick, lightweight test of the entire pipeline to ensure everything is working correctly before launching the full, time-consuming experiments.

# %%
print("\n" + "="*50)
print("🚀 LAUNCHING DRY-RUN EXPERIMENT")
print("="*50)

# We can call the scripts directly using !python
!python -u experiments/run_full_suite.py --task entropy --config experiments/config.yaml --mode dry-run

# %% [markdown]
# ---
# ### Cell 4: Run the Full Experimental Suite
# 
# This is the main execution cell. It runs the full, high-quality experiments for all three pedagogical tasks.
# 
# **Estimated Runtime:** Approximately 1 to 1.5 hours on a T4 GPU.

# %%
print("\n" + "="*50)
print("🚀 LAUNCHING FULL EXPERIMENTAL SUITE")
print("This will take approximately 1 to 1.5 hours.")
print("="*50)

# --- Run for Entropy ---
print("\n--- Running FULL experiment for ENTROPY ---")
!python -u experiments/run_full_suite.py --task entropy --config experiments/config.yaml --mode full

# --- Run for D-Day ---
print("\n--- Running FULL experiment for D-DAY ---")
!python -u experiments/run_full_suite.py --task d-day --config experiments/config.yaml --mode full

# --- Run for Euler's Identity ---
print("\n--- Running FULL experiment for EULERS_IDENTITY ---")
!python -u experiments/run_full_suite.py --task eulers_identity --config experiments/config.yaml --mode full

print("\n🎉 Full experimental suite complete.")

# %% [markdown]
# ---
# ### Cell 5: Analyze Results and Display Summary
# 
# This final cell runs the analysis script on the generated results and displays the final summary table.

# %%
import pandas as pd

print("\n" + "="*50)
print("📊 ANALYZING RESULTS")
print("="*50)

# --- Run the analysis script ---
summary_csv_path = "final_summary_results.csv"
!python -u experiments/analyze_results.py --output {summary_csv_path}


# --- Display the final summary table ---
print("\n--- Final Aggregated Summary ---")
if os.path.exists(summary_csv_path):
    summary_df = pd.read_csv(summary_csv_path)
    # Display the full dataframe without truncation
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
        print(summary_df)
    
    print(f"\n✅ Analysis complete. Results table saved to {summary_csv_path}")
    print("You can download this file from the Colab file browser.")
else:
    print(f"ERROR: Could not find the summary file at {summary_csv_path}")